In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Data preprocessing with diverse dataset types


## Overview
This notebook simplifies [get_started_bq_datasets](https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/datasets/get_started_bq_datasets.ipynb) in Google manual site. Use the original notebook to get more detailed information about this process. Learn more about [BigQuery datasets](https://cloud.google.com/bigquery/docs/datasets-intro) and [Vertex AI for BigQuery users](https://cloud.google.com/vertex-ai/docs/beginner/bqml).

### Dataset

* The dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). In this version of the dataset you consider the fields year, month and day to predict the value of mean daily temperature (mean_temp).

## Install Vertex AI SDK

In [ ]:
! pip install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-bigquery \
                                 tensorflow \
                                 tensorflow-io \
                                 xgboost \
                                 numpy \
                                 pandas \
                                 pyarrow \
                                 db-dtypes

## Configuration

### Authenticate your notebook environment

In [ ]:
import sys
from IPython.display import Markdown, display

PROJECT_ID="ai-hangsik"
LOCATION="us-central1"

# For only colab user, no need this process for Colab Enterprise in Vertex AI.
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

# set project.
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Initialize Vertex AI SDK

In [ ]:
from google.cloud import aiplatform, bigquery
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### Create a Cloud Storage bucket

In [ ]:
# Create a bucket.
BUCKET_URI = f"gs://mlops-{PROJECT_ID}-1207"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-ai-hangsik-1207/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-ai-hangsik-1207' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Import libraries and define constants

In [ ]:
import pandas as pd
import xgboost as xgb
from google.cloud import bigquery

## Create DataSet from BigQuery

### Create BigQuery client


In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

### Create the dataset


In [ ]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.TabularDataset
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.TabularDataset#google_cloud_aiplatform_TabularDataset_create

dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data - bq dataset",
    bq_source=[IMPORT_FILE],
    labels={"user_metadata": BUCKET_URI[5:]},
)

label_column = "mean_temp"

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/721521243942/locations/us-central1/datasets/7610287873593442304/operations/2263486969453477888
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/721521243942/locations/us-central1/datasets/7610287873593442304
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/721521243942/locations/us-central1/datasets/7610287873593442304')


projects/721521243942/locations/us-central1/datasets/7610287873593442304


In [ ]:
print(type(dataset))

<class 'google.cloud.aiplatform.datasets.tabular_dataset.TabularDataset'>


## Create DataSet from GCS(Google Cloud Storage)

### Copy the dataset to Cloud Storage

make a copy of the BigQuery table as a CSV file, to Cloud Storage using the BigQuery extract command.[BigQuery command line interface](https://cloud.google.com/bigquery/docs/reference/bq-cli-reference).

#### Create CSV files into GCS

In [ ]:
comps = BQ_TABLE.split(".")
BQ_PROJECT_DATASET_TABLE = comps[0] + ":" + comps[1] + "." + comps[2]

! bq --location=us extract --destination_format CSV $BQ_PROJECT_DATASET_TABLE $BUCKET_URI/dataset/csv/mydata*.csv

Waiting on bqjob_r7685f3a55d0c4eed_00000193a39f3684_1 ... (66s) Current status: DONE   
['gs://mlops-ai-hangsik-1207/mydata000000000097.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000098.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000099.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000100.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000101.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000102.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000103.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000104.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000105.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000106.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000107.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000108.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000109.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000110.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000111.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000112.csv', 'gs://mlops-ai-hangsik-1207/mydata000000000113.csv', 'gs://mlop

#### Check exported CSV files list

In [ ]:
IMPORT_FILES = ! gsutil ls $BUCKET_URI/dataset/csv/mydata*.csv
print(IMPORT_FILES)

['gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000000.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000001.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000002.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000003.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000004.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000005.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000006.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000007.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000008.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000009.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000010.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000011.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000012.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000013.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000014.csv', 'gs://mlops-ai-hangsik-1

#### Check the contents of a CSV file

In [ ]:
EXAMPLE_FILE = IMPORT_FILES[0]
! gsutil cat $EXAMPLE_FILE | head

station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
39800,99999,1929,12,11,45.5,4,43.5,4,981.4000244140625,4,,,4.3000001907348633,4,19.799999237060547,4,29.899999618530273,,34,false,,,,,false,false,false,false,false,false
37770,99999,1929,12,6,47,4,41.299999237060547,4,993.0999755859375,4,,,4.3000001907348633,4,14.300000190734863,4,18.100000381469727,,45,false,,,,,false,false,false,false,false,false
31590,99999,1929,12,6,45.799999237060547,4,38.299999237060547,4,974.5,4,,,12.399999618530273,4,24.5,4,36.900001525878906,,43,false,,,0,,fa

### Create the dataset

Learn more about [TabularDataset from CSV files](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_gcs_sample-python)

In [ ]:
gcs_source = IMPORT_FILES

print(f"gcs_source : {gcs_source}")

# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.TabularDataset
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.TabularDataset#google_cloud_aiplatform_TabularDataset_create

dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data - csv dataset",
    gcs_source=gcs_source,
    labels={"user_metadata": BUCKET_URI[5:]},
)

label_column = "mean_temp"

print(dataset.resource_name)

gcs_source : ['gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000000.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000001.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000002.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000003.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000004.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000005.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000006.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000007.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000008.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000009.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000010.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000011.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000012.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000013.csv', 'gs://mlops-ai-hangsik-1207/dataset/csv/mydata000000000014.csv', 'gs://mlops

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/721521243942/locations/us-central1/datasets/377506872036425728/operations/1988767392183877632
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/721521243942/locations/us-central1/datasets/377506872036425728
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/721521243942/locations/us-central1/datasets/377506872036425728')


projects/721521243942/locations/us-central1/datasets/377506872036425728


## Read the BigQuery dataset into a tf.data.Dataset

*  [BigQuery TensorFlow reader](https://www.tensorflow.org/io/tutorials/bigquery).
*  [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient

feature_names = "station_number,year,month,day".split(",")
target_name = "mean_temp"

def read_bigquery(project, dataset, table):

    # https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient
    tensorflow_io_bigquery_client = BigQueryClient()
    read_session = tensorflow_io_bigquery_client.read_session(
        parent="projects/" + PROJECT_ID,
        project_id=project,
        dataset_id=dataset,
        table_id=table,
        selected_fields=feature_names + [target_name],
        output_types=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
        requested_streams=2,
    )

    dataset = read_session.parallel_read_rows()
    return dataset


PROJECT, DATASET, TABLE = IMPORT_FILE.split("/")[-1].split(".")
tf_dataset = read_bigquery(PROJECT, DATASET, TABLE)

print(tf_dataset.take(1))

NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZN3tsl8str_util9LowercaseB5cxx11ESt17basic_string_viewIcSt11char_traitsIcEE']

### Read CSV files into a tf.data.Dataset

Alternatively, when your data is in CSV files, you can load the dataset into a tf.data.Dataset using `tf.data.experimental.CsvDataset`, with the following parameters:

- `filenames`: A list of one or more CSV files.
- `header`: Whether CSV file(s) contain a header.
- `select_cols`: Subset of fields (columns) to return.
- `record_defaults`: The output types of the corresponding fields.

Learn more about [tf.data CsvDataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/CsvDataset)

In [ ]:
import tensorflow as tf

feature_names = ["station_number,year,month,day".split(",")]

target_name = "mean_temp"

tf_dataset = tf.data.experimental.CsvDataset(
    filenames=IMPORT_FILES,
    header=True,
    select_cols=feature_names.append(target_name),
    record_defaults=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
)

print(tf_dataset.take(1))

<_TakeDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>


### Create a BigQuery dataset from a pandas dataframe

You can create a BigQuery dataset from a pandas dataframe using the BigQuery `create_dataset()` and `load_table_from_dataframe()` methods, as follows:

- `create_dataset()`: Creates an empty BigQuery dataset, with the following parameters:
 - `dataset_ref`: The `DatasetReference` created from the dataset_id -- e.g., samples.
- `load_table_from_dataframe()`: Loads one or more CSV files into a table within the corresponding dataset, with the following parameters:
 - `dataframe`: The dataframe.
 - `table`: The `TableReference` for the table.
 - `job_config`: Specifications on how to load the dataframe data.

In [ ]:
LOCATION = "us"

SCHEMA = [
    bigquery.SchemaField("station_number", "STRING"),
    bigquery.SchemaField("year", "INTEGER"),
    bigquery.SchemaField("month", "INTEGER"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("mean_temp", "FLOAT"),
]


DATASET_ID = "samples"
TABLE_ID = "gsod"


def create_bigquery_dataset(dataset_id):
    dataset = bigquery.Dataset(
        bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id)
    )
    dataset.location = "us"

    try:
        dataset = bqclient.create_dataset(dataset)  # API request
        return True
    except Exception as err:
        print(err)
        if err.code != 409:  # http_client.CONFLICT
            raise
    return False


def load_data_into_bigquery(dataframe, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig(
        # Specify a (partial) schema. All columns are always written to the
        # table. The schema is used to assist in data type definitions.
        schema=[
            bigquery.SchemaField("station_number", "STRING"),
            bigquery.SchemaField("year", "INTEGER"),
            bigquery.SchemaField("month", "INTEGER"),
            bigquery.SchemaField("day", "INTEGER"),
            bigquery.SchemaField("mean_temp", "FLOAT"),
        ],
        # Optionally, set the write disposition. BigQuery appends loaded rows
        # to an existing table by default, but with WRITE_TRUNCATE write
        # disposition it replaces the table with the loaded data.
        write_disposition="WRITE_TRUNCATE",
    )

    NEW_BQ_TABLE = f"{PROJECT_ID}.{dataset_id}.{table_id}"

    job = bqclient.load_table_from_dataframe(
        dataframe, NEW_BQ_TABLE, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = bqclient.get_table(NEW_BQ_TABLE)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), NEW_BQ_TABLE
        )
    )


load_data_into_bigquery(dataframe, DATASET_ID, TABLE_ID)

### Create a BigQuery dataset from CSV files

You can create a BigQuery dataset from CSV files using the BigQuery `create_dataset()` and `load_table_from_uri()` methods, as follows:

- `create_dataset()`: Creates an empty BigQuery dataset, with the following parameters:
 - `dataset_ref`: The `DatasetReference` created from the dataset_id -- e.g., samples.
- `load_table_from_uri()`: Loads one or more CSV files into a table within the corresponding dataset, with the following parameters:
 - `url`: A set of one or more CVS files in Cloud Storage storage.
 - `table`: The `TableReference` for the table.
 - `job_config`: Specifications on how to load the CSV data.

Learn more about [Importing CSV data into BigQuery](https://www.tensorflow.org/io/tutorials/bigquery#import_census_data_into_bigquery).

In [ ]:
LOCATION = "us"

CSV_SCHEMA = [
    bigquery.SchemaField("station_number", "STRING"),
    bigquery.SchemaField("wban_number", "STRING"),
    bigquery.SchemaField("year", "INTEGER"),
    bigquery.SchemaField("month", "INTEGER"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("mean_temp", "FLOAT"),
    bigquery.SchemaField("num_mean_temp_samples", "INTEGER"),
    bigquery.SchemaField("mean_dew_point", "FLOAT"),
    bigquery.SchemaField("num_mean_dew_point_samples", "INTEGER"),
    bigquery.SchemaField("mean_sealevel_pressure", "FLOAT"),
    bigquery.SchemaField("num_mean_sealevel_pressure_samples", "INTEGER"),
    bigquery.SchemaField("mean_station_pressure", "FLOAT"),
    bigquery.SchemaField("num_mean_station_pressure_samples", "INTEGER"),
    bigquery.SchemaField("mean_visibility", "FLOAT"),
    bigquery.SchemaField("num_mean_visibility_samples", "INTEGER"),
    bigquery.SchemaField("mean_wind_speed", "FLOAT"),
    bigquery.SchemaField("num_mean_wind_speed_samples", "INTEGER"),
    bigquery.SchemaField("max_sustained_wind_speed", "FLOAT"),
    bigquery.SchemaField("max_gust_wind_speed", "FLOAT"),
    bigquery.SchemaField("max_temperature", "FLOAT"),
    bigquery.SchemaField("max_temperature_explicit", "BOOLEAN"),
    bigquery.SchemaField("min_temperature", "FLOAT"),
    bigquery.SchemaField("min_temperature_explicit", "BOOLEAN"),
    bigquery.SchemaField("total_percipitation", "FLOAT"),
    bigquery.SchemaField("snow_depth", "FLOAT"),
    bigquery.SchemaField("fog", "BOOLEAN"),
    bigquery.SchemaField("rain", "BOOLEAN"),
    bigquery.SchemaField("snow", "BOOLEAN"),
    bigquery.SchemaField("hail", "BOOLEAN"),
    bigquery.SchemaField("thunder", "BOOLEAN"),
    bigquery.SchemaField("tornado", "BOOLEAN"),
]


DATASET_ID = "samples"
TABLE_ID = "gsod"


def load_data_into_bigquery(url, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.schema = CSV_SCHEMA
    job_config.skip_leading_rows = 1  # heading

    load_job = bqclient.load_table_from_uri(url, table, job_config=job_config)
    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = bqclient.get_table(table)
    print("Loaded {} rows.".format(destination_table.num_rows))


load_data_into_bigquery(IMPORT_FILES, DATASET_ID, TABLE_ID)

### Read BigQuery table into XGboost DMatrix

Currently, there is no direct data feeding connector between BigQuery and the open source XGBoost. The BigQuery ML service has a built-in XGBoost training module.

Alernatively, you extract the data either as a pandas dataframe or as CSV files. The extracted data is then given as an input to a `DMatrix` object when training the model.

Learn more about [Getting started with built-in XGBoost](https://cloud.google.com/ai-platform/training/docs/algorithms/xgboost-start).

### Read pandas table into XGboost DMatrix

Next, you load the pandas dataframe into a `DMatrix` object. XGBoost does not support non-numeric inputs. Any column that is categorical need to be one-hot encoded prior to loading the dataframe.

In [ ]:
dataframe["station_number"] = pd.to_numeric(dataframe["station_number"])
labels = dataframe["mean_temp"]
data = dataframe.drop(["mean_temp"], axis=1)

dtrain = xgb.DMatrix(data, label=labels)

### Read CSV files into XGboost DMatrix

Currently, there is no Cloud Storage support in XGBoost. If you use CSV files for input, you need to download them locally.

In [ ]:
! gsutil cp $EXAMPLE_FILE data.csv

dtrain = xgb.DMatrix("data.csv?format=csv&label_column=4")

### Useful BigQuery Operations

#### Create a view of the BigQuery dataset

Alternatively, you can create a logical view of a BigQuery dataset that has a subset of the fields.

Learn more about [Creating BigQuery views](https://cloud.google.com/bigquery/docs/views).

In [ ]:
# Set dataset name and view name in BigQuery
BQ_MY_DATASET = "[your-dataset-name]"
BQ_MY_TABLE = "[your-view-name]"

# Otherwise, use the default names
if (
    BQ_MY_DATASET == ""
    or BQ_MY_DATASET is None
    or BQ_MY_DATASET == "[your-dataset-name]"
):
    BQ_MY_DATASET = "mlops_dataset"

if BQ_MY_TABLE == "" or BQ_MY_TABLE is None or BQ_MY_TABLE == "[your-view-name]":
    BQ_MY_TABLE = "mlops_view"

In [ ]:
# Create the resources
! bq --location=US mk -d \
$PROJECT_ID:$BQ_MY_DATASET

sql_script = f'''
CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_MY_DATASET}.{BQ_MY_TABLE}`
AS SELECT station_number,year,month,day,mean_temp FROM `{BQ_TABLE}`
'''
print(sql_script)

query = bqclient.query(sql_script)

#### Read the BigQuery dataset into a pandas dataframe

Next, you read a sample of the dataset into a pandas dataframe using BigQuery `list_rows()` and `to_dataframe()` method, as follows:

- `list_rows()`: Performs a query on the specified table and returns a row iterator to the query results. Optionally specify:
 - `selected_fields`: Subset of fields (columns) to return.
 - `max_results`: The maximum number of rows to return. Same as SQL LIMIT command.


- `rows.to_dataframe()`: Invokes the row iterator and reads in the data into a pandas dataframe.

Learn more about [Loading BigQuery table into a dataframe](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(BQ_TABLE)

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
)

dataframe = rows.to_dataframe()
print(dataframe.head())

  station_number  year  month  day  mean_temp
0          39800  1929     12   11  45.500000
1          37770  1929     12    6  47.000000
2          31590  1929     12    6  45.799999
3          30910  1929     11   25  49.799999
4          33790  1929      8   29  62.000000


### Clean up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Vertex AI dataset resource
- Cloud Storage Bucket
- BigQuery dataset

Set `delete_storage` to _True_ to delete the storage resources used in this notebook.

In [ ]:
import os

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the temporary BigQuery dataset
! bq rm -r -f $PROJECT_ID:$DATASET_ID

delete_storage = False
if delete_storage or os.getenv("IS_TESTING"):
    # Delete the created GCS bucket
    ! gsutil rm -r $BUCKET_URI
    # Delete the created BigQuery datasets
    ! bq rm -r -f $PROJECT_ID:$BQ_MY_DATASET